# **RNN Model Notebook**

@authors: miguelrocha and Grupo 03

In [ ]:
# Notebook Imports
import numpy as np
import pandas as pd
import re
from collections import Counter
import pickle
import random
import time
import os
import requests
import zipfile

from helpers.dataset import Dataset
from helpers.activation import TanhActivation
from helpers.losses import BinaryCrossEntropy
from helpers.metrics import accuracy
from helpers.activation import ReLUActivation
from models.rnn_model import RNN

**Modificação na classe Optimizer**

In [ ]:
class Optimizer:
    def __init__(self, learning_rate=0.01, momentum=0.9):
        self.velocity = {}  # Dicionário para armazenar velocidades dos gradientes
        self.learning_rate = learning_rate
        self.momentum = momentum

    def update(self, param, grad):
        """Atualiza os pesos usando Gradient Descent com Momentum"""

        param_id = id(param)  # 🔹 Usar ID único do numpy array

        if param_id not in self.velocity:
            self.velocity[param_id] = np.zeros_like(grad)

        # Atualização com momentum
        self.velocity[param_id] = self.momentum * self.velocity[param_id] + (1 - self.momentum) * grad
        return param - self.learning_rate * self.velocity[param_id]  # 🔹 Retorna os novos pesos



### **Tratamento de Dados**

**Análise Inicial dos Datasets e Junção dos mesmos para tratamento simultâneo**

In [ ]:
# Definir os caminhos dos arquivos de TREINO
input_csv1 = "../tarefa_1/clean_input_datasets/gpt_vs_human_data_set_inputs.csv"
output_csv1 = "../tarefa_1/clean_output_datasets/gpt_vs_human_data_set_outputs.csv"

# Definir os caminhos dos arquivos de TESTE FINAL
input_csv2 = "classify_input_datasets/dataset2_inputs.csv"
output_csv2 = "../tarefa_2/layout_datasets/dataset2_layout_outputs.csv" # dataset apenas utilizado para adicionar o layout ID Label
 
# Carregar os datasets de treino
df_input1 = pd.read_csv(input_csv1, sep="\t")  
df_output1 = pd.read_csv(output_csv1, sep="\t")

# Carregar os datasets de teste
df_input2 = pd.read_csv(input_csv2, sep="\t")
df_output2 = pd.read_csv(output_csv2, sep="\t")

# Junção com coluna ID
df_train = pd.merge(df_input1, df_output1, on="ID")
df_test = pd.merge(df_input2, df_output2, on="ID")

# Concatenar treino e teste para aplicar as alterações simultaneamente
df_dataset1_merged = pd.concat([df_train, df_test], ignore_index=True)

# Mostrar as primeiras 5 linhas do dataset completo
print("\nDataset Completo - Primeiras 5 linhas:")
print(df_dataset1_merged.head())

print("\nDataset Completo - Ultimas 5 linhas:")
print(df_dataset1_merged.tail())

**Remover caracteres especiais e pontuação e Converter em minúsculas**

In [ ]:
# Função para limpar texto
def clean_text(text):
    text = text.lower()  # Converter para minúsculas
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # Remover pontuação
    return text

df_dataset1_merged["clean_text"] = df_dataset1_merged["Text"].apply(clean_text)

# Manter apenas as colunas desejadas e renomear clean_text para Text
df_dataset1_merged = df_dataset1_merged[["ID", "clean_text", "Label"]].rename(columns={"clean_text": "Text"})

print("Texto limpo - primeiras 5 linhas:")
print(df_dataset1_merged.head())

**Remover stopwords**

In [ ]:
# Lista de stopwords comuns
stopwords = {
    "the", "of", "and", "in", "to", "is", "a", "that", "for", "are", "on", "with", 
    "as", "at", "by", "from", "this", "it", "an", "be", "or", "which", "was", "were"
}

# Função para remover stopwords
def remove_stopwords(text):
    words = text.split()  # Dividir em palavras
    filtered_words = [word for word in words if word not in stopwords]  # Remover stopwords
    return " ".join(filtered_words)  # Juntar as palavras de novo

# Aplicar ao dataset
df_dataset1_merged["Text"] = df_dataset1_merged["Text"].apply(remove_stopwords)

# Exibir as primeiras 5 linhas após remoção de stopwords
print("Texto sem stopwords - primeiras 5 linhas:")
print(df_dataset1_merged.head())



**Criar Embeddings e Label Encoding**

In [ ]:
# Mapear labels para valores numéricos
label_map = {"Human": 0, "AI": 1}
df_dataset1_merged["Label"] = df_dataset1_merged["Label"].map(label_map)

# Carregar o GloVe
EMBEDDING_DIM = 50  # Dimensão do embedding

# Diretório e nome do ficheiro GloVe
glove_dir = "helpers"
glove_filename = "glove.6B.50d.txt"
glove_zip_url = "http://nlp.stanford.edu/data/glove.6B.zip"  # URL do GloVe oficial

# Criar diretório se não existir
os.makedirs(glove_dir, exist_ok=True)

# Caminho completo do ficheiro
glove_path = os.path.join(glove_dir, glove_filename)
glove_zip_path = os.path.join(glove_dir, "glove.6B.zip")

# Verificar se o ficheiro já existe
if not os.path.exists(glove_path):
    print("Ficheiro GloVe não encontrado. A fazer download...")

    # Download do ficheiro ZIP do GloVe
    response = requests.get(glove_zip_url, stream=True)
    with open(glove_zip_path, "wb") as f:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)
    
    print("Download concluído. A extrair ficheiros...")

    # Extrair apenas o ficheiro necessário
    with zipfile.ZipFile(glove_zip_path, "r") as zip_ref:
        zip_ref.extract(glove_filename, path=glove_dir)

    print("Extração concluída!")

# Agora podemos carregar o GloVe
embedding_dict = {}
with open(glove_path, "r", encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.array(values[1:], dtype="float32")
        embedding_dict[word] = vector

print(f"Total de palavras carregadas do GloVe: {len(embedding_dict)}")

# Converter palavras para embeddings
def text_to_embedding(text, embedding_dict, embedding_dim=50):
    words = text.split()
    embeddings = [embedding_dict.get(word, np.zeros(embedding_dim)) for word in words]  # Usa vetor do GloVe ou vetor zerado
    
    # Se a lista estiver vazia, retorna um vetor de zeros
    if len(embeddings) == 0:
        embeddings = [np.zeros(embedding_dim)]

    return embeddings

df_dataset1_merged["Embedding"] = df_dataset1_merged["Text"].apply(lambda x: text_to_embedding(x, embedding_dict, EMBEDDING_DIM))


**Padronizar o comprimento das sequências**


In [ ]:
# Padronizar comprimento das sequências
MAX_SEQUENCE_LENGTH = 130  # foram testados vários valores sendo o melhor 130

def pad_embedding_sequence(seq, max_length, embedding_dim):
    seq = np.array(seq)  # Garante que a sequência é um array NumPy
    
    if seq.shape[0] == 0:  # Se for uma sequência vazia, criar um array de zeros
        seq = np.zeros((1, embedding_dim))

    if seq.shape[0] > max_length:  # Truncar se for maior
        return seq[:max_length]
    
    padding = np.zeros((max_length - seq.shape[0], embedding_dim))  # Criar padding
    return np.vstack([seq, padding])  # Adicionar padding no final

# Aplicar padding às sequências de embeddings
df_dataset1_merged["Embedding"] = df_dataset1_merged["Embedding"].apply(
    lambda x: pad_embedding_sequence(x, MAX_SEQUENCE_LENGTH, EMBEDDING_DIM)
)

# Converter para array NumPy para alimentar o modelo
X = np.array(df_dataset1_merged["Embedding"].tolist())
y = np.array(df_dataset1_merged["Label"])  # Labels numéricos

print("Formato final dos dados para o modelo:", X.shape)  # Deve ser (n_amostras, MAX_SEQUENCE_LENGTH, EMBEDDING_DIM)

# Manter apenas as colunas desejadas e renomear "Embedding" para "Text"
df_dataset1_merged = df_dataset1_merged[["ID", "Embedding", "Label"]].rename(columns={"Embedding": "Text"})

print("Dataset após embedding - primeiras 5 linhas:")
print(df_dataset1_merged.head())


**Normalização dos Embeddings**


In [ ]:
# Função para normalizar cada embedding (zero mean, unit variance)
def normalize_embedding(emb):
    mean = np.mean(emb, axis=0)  # Média por dimensão do embedding
    std = np.std(emb, axis=0) + 1e-8  # Desvio padrão (evita divisão por zero)
    return (emb - mean) / std

# Aplicar normalização alternativa aos embeddings
df_dataset1_merged["Text"] = df_dataset1_merged["Text"].apply(normalize_embedding)

# Converter para array NumPy para treinar o modelo
X = np.array(df_dataset1_merged["Text"].tolist())
y = np.array(df_dataset1_merged["Label"])  # Labels numéricos

print("Formato final dos dados para o modelo:", X.shape)  # Deve ser (n_amostras, MAX_SEQUENCE_LENGTH, EMBEDDING_DIM)

# Print do dataset atualizado
print("\nDataset após normalização dos embeddings:")
print(df_dataset1_merged.head())


**Drop da coluna ID**

In [ ]:
if "ID" in df_dataset1_merged.columns:
    df_dataset1_merged = df_dataset1_merged.drop(columns=["ID"])

print("Formato final dos dados para o modelo:", X.shape)  # Deve ser (n_amostras, MAX_SEQUENCE_LENGTH, EMBEDDING_DIM)

# Print do dataset atualizado
print("\nDataset após drop:")
print(df_dataset1_merged.head())

**Divisão do Dataset**

Dataset de Treino:

- 70% : Treino
- 15% : Validação
- 15% : Teste

Dataset de Avaliação:

- 100% : Teste Final


In [ ]:
# Definir seed global para garantir reprodutibilidade
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

######################################################### dataset de teste
# Separar as últimas linhas para avaliação final
df_eval_final = df_dataset1_merged.tail(100)

# Remover essas linhas do dataset antes de embaralhar
df_remaining = df_dataset1_merged.iloc[:-100]
#########################################################

# Embaralhar o dataset restante
df_remaining = df_remaining.sample(frac=1, random_state=SEED).reset_index(drop=True)

# Definir proporções de treino (70%), validação (15%) e teste (15%)
train_ratio = 0.7
val_ratio = 0.15  # 15% validação
test_ratio = 0.15  # 15% teste

# Definir índices para divisão
train_index = int(len(df_remaining) * train_ratio)
val_index = train_index + int(len(df_remaining) * val_ratio)

# Separar os conjuntos de treino, validação e teste
df_train = df_remaining.iloc[:train_index]
df_val = df_remaining.iloc[train_index:val_index]
df_test = df_remaining.iloc[val_index:]

# Print dos tamanhos dos datasets
print(f"Tamanho do conjunto de treino: {df_train.shape}")
print(f"Tamanho do conjunto de validação: {df_val.shape}")
print(f"Tamanho do conjunto de teste: {df_test.shape}")
print(f"Tamanho do conjunto de avaliação final: {df_eval_final.shape}")

# Converter para arrays NumPy
X_train, y_train = np.array(df_train["Text"].tolist()), np.array(df_train["Label"])
X_val, y_val = np.array(df_val["Text"].tolist()), np.array(df_val["Label"])
X_test, y_test = np.array(df_test["Text"].tolist()), np.array(df_test["Label"])
X_eval_final, y_eval_final = np.array(df_eval_final["Text"].tolist()), np.array(df_eval_final["Label"])

# Print dos formatos dos dados
print(f"Formato dos dados:")
print(f"   Treino: {X_train.shape}")
print(f"   Validação: {X_val.shape}")
print(f"   Teste: {X_test.shape}")
print(f"   Avaliação final: {X_eval_final.shape}")



**Verificação Final do Dataset**

In [ ]:
print("\n Primeiras 5 entradas do conjunto de TREINO:")
print(df_train.head())

print("\n Primeiras 5 entradas do conjunto de VALIDAÇÃO:")
print(df_val.head())

print("\n Primeiras 5 entradas do conjunto de TESTE:")
print(df_test.head())

print("\n Primeiras 5 entradas do conjunto de AVALIAÇÃO FINAL:")
print(df_eval_final.head())


### **Construção do modelo RNN com código raiz (Sem TensorFlow/SKLearn)**

**Inicialização de Pesos**

Antes de tudo, vamos definir os pesos da rede:

- W_xh: Pesa a entrada para os neurônios recorrentes.
- W_hh: Pesa as conexões recorrentes.
- W_hy: Pesa a saída do neurônio recorrente para a previção final.
- b_h e b_y: Bias da camada oculta e da saída.

In [ ]:
# Definir hiperparâmetros
input_size = 50    # Dimensão dos embeddings
hidden_size = 64   # Número de neurônios na camada oculta
output_size = 1    # Saída binária (0 ou 1)
learning_rate = 0.01  

# Inicializar pesos
np.random.seed(42)  # Para reprodutibilidade
W_xh = np.random.randn(input_size, hidden_size) * 0.01  # Pesos da entrada para a camada oculta
W_hh = np.random.randn(hidden_size, hidden_size) * 0.01 # Pesos da camada oculta para ela mesma
W_hy = np.random.randn(hidden_size, output_size) * 0.01 # Pesos da camada oculta para saída

# Bias
b_h = np.zeros((1, hidden_size))
b_y = np.zeros((1, output_size))

print("Pesos e Biases inicializados!")

**Função de Custo (Binary Cross-Entropy)**

In [ ]:
def binary_cross_entropy(y_true, y_pred):
    y_pred = np.clip(y_pred, 1e-8, 1 - 1e-8)  # 🔹 Evita log(0) ou log(1)
    return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred)) / y_pred.shape[0]

**Mini-Batches**

In [ ]:
def get_mini_batches(X, y, batch_size=16, shuffle=True):
    """Divide os dados em mini-batches."""
    n_samples = X.shape[0]
    indices = np.arange(n_samples)
    if shuffle:
        np.random.shuffle(indices)
    
    for start in range(0, n_samples, batch_size):
        end = min(start + batch_size, n_samples)
        yield X[indices[start:end]], y[indices[start:end]]


**Otimização de Hiperparâmetros (Inicial)**

In [ ]:
# Função de ativação Sigmoid
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Definir pesos corretamente (Xavier Initialization)
W_xh = np.random.randn(input_size, hidden_size) * np.sqrt(1. / input_size)
W_hh = np.random.randn(hidden_size, hidden_size) * np.sqrt(1. / hidden_size)
W_hy = np.random.randn(hidden_size, output_size) * np.sqrt(1. / hidden_size)

HYPERPARAMS = [
    {"epochs": 5, "batch_size": 8, "learning_rate": 0.01, "momentum": 0.9, "bptt_trunc": 2},
    {"epochs": 10, "batch_size": 16, "learning_rate": 0.005, "momentum": 0.95, "bptt_trunc": 3},
    {"epochs": 7, "batch_size": 8, "learning_rate": 0.007, "momentum": 0.8, "bptt_trunc": 2},
]

best_accuracy = 0
best_params = None
best_model = None

# Testando hiperparâmetros
for params in HYPERPARAMS:
    print(f"\nTestando hiperparâmetros: {params}")

    rnn = RNN(
        n_units=20,
        # activation=ReLUActivation(),
        activation=TanhActivation(),
        bptt_trunc=params["bptt_trunc"],
        input_shape=(MAX_SEQUENCE_LENGTH, EMBEDDING_DIM),
        epochs=params["epochs"],
        batch_size=params["batch_size"],
        learning_rate=params["learning_rate"],
        momentum=params["momentum"],
        loss=BinaryCrossEntropy,
        metric=accuracy
    )

    optimizer = Optimizer(learning_rate=params["learning_rate"])
    rnn.initialize(optimizer)

    for epoch in range(params["epochs"]):
        total_loss = 0
        for X_batch, y_batch in get_mini_batches(X_train, y_train, params["batch_size"]):
            y_pred = rnn.forward_propagation(X_batch)
            y_pred_final = sigmoid(y_pred[:, -1, :])  # Aplica Sigmoid na última saída

            loss = binary_cross_entropy(y_batch.reshape(-1, 1), y_pred_final)
            grad_loss = (y_pred_final - y_batch.reshape(-1, 1)) / y_batch.shape[0]

            grad_loss_expanded = np.zeros_like(y_pred)
            grad_loss_expanded[:, -1, :] = grad_loss

            rnn.backward_propagation(grad_loss_expanded)

            total_loss += loss

        print(f"Época {epoch+1}/{params['epochs']} - Loss: {total_loss:.4f}")

    # Avaliação
    preds = rnn.predict(X_val)
    
    # Debug do formato de `preds`
    print(f"Formato de preds: {preds.shape}")

    # Corrigir caso `preds` seja 1D
    if preds.ndim == 1:
        preds = preds[:, np.newaxis]

    acc = accuracy(y_val, preds)

    print(f"Accuracy com esses hiperparâmetros: {acc:.4f}")

    if acc > best_accuracy:
        best_accuracy = acc
        best_params = params
        best_model = rnn

print(f"\nMelhor combinação encontrada: {best_params} com accuracy {best_accuracy:.4f}")

**Treinar o Modelo Final com melhor accuracy (obtido no passo anterior)**

In [ ]:
final_rnn = RNN(
    n_units=20,
    # activation=ReLUActivation(),
    activation=TanhActivation(),
    bptt_trunc=best_params["bptt_trunc"],
    input_shape=(MAX_SEQUENCE_LENGTH, EMBEDDING_DIM),
    epochs=best_params["epochs"],
    batch_size=best_params["batch_size"],
    learning_rate=best_params["learning_rate"],
    momentum=best_params["momentum"],
    loss=BinaryCrossEntropy,
    metric=accuracy
)

final_optimizer = Optimizer(learning_rate=best_params["learning_rate"])
final_rnn.initialize(final_optimizer)

for epoch in range(best_params["epochs"]):
    total_loss = 0
    for X_batch, y_batch in get_mini_batches(X_train, y_train, best_params["batch_size"]):
        y_pred = final_rnn.forward_propagation(X_batch)
        y_pred_final = sigmoid(y_pred[:, -1, :])  # Aplica Sigmoid na última saída

        loss = binary_cross_entropy(y_batch.reshape(-1, 1), y_pred_final)

        # Calcular o gradiente correto
        grad_loss = (y_pred_final - y_batch.reshape(-1, 1)) / y_batch.shape[0]

        # Expandir para 3 dimensões para ser compatível com a RNN
        grad_loss_expanded = np.zeros_like(y_pred)  # (batch_size, timesteps, output_size)
        grad_loss_expanded[:, -1, :] = grad_loss  # Apenas o último timestep recebe gradiente

        # Passar o gradiente expandido
        final_rnn.backward_propagation(grad_loss_expanded)

        total_loss += loss

    print(f"Treino final - Época {epoch+1}/{best_params['epochs']} - Loss: {total_loss:.4f}")

# Testar Modelo Final
y_test_pred = final_rnn.predict(X_test)

print(f"Formato de y_test_pred: {y_test_pred.shape}")  # 🛠️ Debug

# Se for 1D, expandimos para 2D
if y_test_pred.ndim == 1:
    y_test_pred = y_test_pred[:, np.newaxis]

# Se for 2D (batch_size, timesteps), pegamos o último timestep
if y_test_pred.ndim == 2:
    y_test_pred_final = y_test_pred[:, -1]  #  Sem `:` no final, pois já é 1D
else:
    y_test_pred_final = y_test_pred[:, -1, :]  #  Apenas se for 3D

y_test_pred_labels = (y_test_pred_final > 0.5).astype(int)

y_test_true = y_test.flatten()
accuracy = np.mean(y_test_pred_labels == y_test_true)
print(f"\nAccuracy final no conjunto de teste: {accuracy:.4f}")

# Criar DataFrame com Expected vs Predicted
df_results = pd.DataFrame({
    "expected_value": y_test_true,
    "predicted_value_raw": y_test_pred_final.flatten(),  # Valor original antes do arredondamento
    "predicted_value": y_test_pred_labels.flatten()  # Valor final binário (0 ou 1)
})

# Mostrar as previsões para comparação
print("\nComparação entre valores esperados e previstos:")
print(df_results)



**Previsão para o Dataset2 (disponibilizado pelo professor)**

In [ ]:
# Testar Modelo Final
y_test_pred2 = final_rnn.predict(X_eval_final)

print(f"Formato de y_test_pred2: {y_test_pred2.shape}")  # 🛠️ Debug

# Se for 1D, expandimos para 2D
if y_test_pred2.ndim == 1:
    y_test_pred2 = y_test_pred2[:, np.newaxis]

# Se for 2D (batch_size, timesteps), pegamos o último timestep
if y_test_pred2.ndim == 2:
    y_test_pred_final2 = y_test_pred2[:, -1]  #  Sem `:` no final, pois já é 1D
else:
    y_test_pred_final2 = y_test_pred2[:, -1, :]  #  Apenas se for 3D

y_test_pred_labels2 = (y_test_pred_final2 > 0.5).astype(int)

######################################################################### criação do ficheiro csv com a previsão

# Criar IDs para cada amostra com o formato "D2-1", "D2-2", etc.
id_column = [f"D2-{i}" for i in range(1, len(y_test_pred_labels2) + 1)]

# Converter labels para "Human" e "AI"
labels = np.where(y_test_pred_labels2.flatten() == 1, "AI", "Human")

# Criar DataFrame com ID e LABEL
df_output = pd.DataFrame({
    "ID": id_column,
    "Label": labels
})

# Não estamos a guardar a previsão em .csv visto que este não é o melhor modelo

# Guardar em CSV com separação por tabulação
# df_output.to_csv("rnn_predictions.csv", index=False, sep='\t')

# print("Ficheiro 'rnn_predictions.csv' gerado com sucesso!")


### **Análise de resultados**

**Treino com dataset: gpt_vs_human**

- Durante o treino: 0.87 - 0.9

- Para dataset1: 0.66

- Para dataset2: 0.8 - 1.0

- Para ai_human: 0.51

**Treino com dataset: ai_human**

- Durante o treino: 0.81 - 0.84

- Para gpt_vs_human: 0.49

### **Hypertuning com base no modelo anterior - teste com 3600 combinações diferentes**

Foi feito o loop apresentado abaixo, com 3600 combinações, porém por uma questão de brevidade, estamos neste momento a rodar o código apenas com o melhor resultado obtido:

**Melhor combinação encontrada: {'epochs': 5, 'batch_size': 8, 'learning_rate': 0.01, 'momentum': 0.8, 'bptt_trunc': 6} com accuracy 0.8929**

In [ ]:
print(f"Tipo de accuracy antes da chamada: {type(accuracy)}")
if not callable(accuracy):  # Se não for mais uma função
    del accuracy  # Remover a variável sobrescrita
    from helpers.metrics import accuracy  # Reimporte 


# Definir hiperparâmetros para busca extensa
# HYPERPARAMS = [
#     {"epochs": ep, "batch_size": bs, "learning_rate": lr, "momentum": mo, "bptt_trunc": bt}
#     for ep in [5, 10, 15, 20, 25, 30]
#     for bs in [8, 16, 32, 64]
#     for lr in [0.01, 0.005, 0.001, 0.0005, 0.0001]
#     for mo in [0.7, 0.8, 0.85, 0.9, 0.95, 0.99]
#     for bt in [2, 3, 4, 5, 6]
# ]

# Apenas com os melhores hiperparâmetros calculados anteriormente
HYPERPARAMS = [
    {"epochs": ep, "batch_size": bs, "learning_rate": lr, "momentum": mo, "bptt_trunc": bt}
    for ep in [5]
    for bs in [8]
    for lr in [0.01]
    for mo in [0.8]
    for bt in [6]
]

best_accuracy = 0
best_params = None
best_model = None

start_time = time.time()
MAX_TIME = 21600 #6 horas em segundos

# Teste de hiperparâmetros 
for params in HYPERPARAMS:
    if time.time() - start_time > MAX_TIME:
        break
    
    print(f"\nA testar hiperparâmetros: {params}")
    
    rnn = RNN(
        n_units=20,
        activation=TanhActivation(),
        bptt_trunc=params["bptt_trunc"],
        input_shape=(MAX_SEQUENCE_LENGTH, EMBEDDING_DIM),
        epochs=params["epochs"],
        batch_size=params["batch_size"],
        learning_rate=params["learning_rate"],
        momentum=params["momentum"],
        loss=BinaryCrossEntropy,
        metric=accuracy
    )
    
    optimizer = Optimizer(learning_rate=params["learning_rate"])
    rnn.initialize(optimizer)
    
    for epoch in range(params["epochs"]):
        total_loss = 0
        for X_batch, y_batch in get_mini_batches(X_train, y_train, params["batch_size"]):
            y_pred = rnn.forward_propagation(X_batch)
            y_pred_final = sigmoid(y_pred[:, -1, :])

            loss = binary_cross_entropy(y_batch.reshape(-1, 1), y_pred_final)
            grad_loss = (y_pred_final - y_batch.reshape(-1, 1)) / y_batch.shape[0]
            
            grad_loss_expanded = np.zeros_like(y_pred)
            grad_loss_expanded[:, -1, :] = grad_loss
            
            rnn.backward_propagation(grad_loss_expanded)
            total_loss += loss
        
        print(f"Época {epoch+1}/{params['epochs']} - Loss: {total_loss:.4f}")
    
    preds = rnn.predict(X_val)
    if preds.ndim == 1:
        preds = preds[:, np.newaxis]
    acc_value = accuracy(y_val, preds)
    
    print(f"Accuracy com esses hiperparâmetros: {acc_value:.4f}")
    
    if acc_value > best_accuracy:
        best_accuracy = acc_value
        best_params = params
        best_model = rnn

print(f"\nMelhor combinação encontrada: {best_params} com accuracy {best_accuracy:.4f}")

**Treino do modelo final, com os melhores hiperparâmetros**

In [ ]:
final_rnn = RNN(
    n_units=20,
    activation=TanhActivation(),
    bptt_trunc=best_params["bptt_trunc"],
    input_shape=(MAX_SEQUENCE_LENGTH, EMBEDDING_DIM),
    epochs=best_params["epochs"],
    batch_size=best_params["batch_size"],
    learning_rate=best_params["learning_rate"],
    momentum=best_params["momentum"],
    loss=BinaryCrossEntropy,
    metric=accuracy
)

final_optimizer = Optimizer(learning_rate=best_params["learning_rate"])
final_rnn.initialize(final_optimizer)

for epoch in range(best_params["epochs"]):
    total_loss = 0
    for X_batch, y_batch in get_mini_batches(X_train, y_train, best_params["batch_size"]):
        y_pred = final_rnn.forward_propagation(X_batch)
        y_pred_final = sigmoid(y_pred[:, -1, :])  # Aplica Sigmoid na última saída

        loss = binary_cross_entropy(y_batch.reshape(-1, 1), y_pred_final)

        # Calcular o gradiente correto
        grad_loss = (y_pred_final - y_batch.reshape(-1, 1)) / y_batch.shape[0]

        # Expandir para 3 dimensões para ser compatível com a RNN
        grad_loss_expanded = np.zeros_like(y_pred)  # (batch_size, timesteps, output_size)
        grad_loss_expanded[:, -1, :] = grad_loss  # Apenas o último timestep recebe gradiente

        # Passar o gradiente expandido
        final_rnn.backward_propagation(grad_loss_expanded)

        total_loss += loss

    print(f"Treino final - Época {epoch+1}/{best_params['epochs']} - Loss: {total_loss:.4f}")

# Testar Modelo Final
y_test_pred = final_rnn.predict(X_test)

print(f"Formato de y_test_pred: {y_test_pred.shape}")  # Debug

# Se for 1D, expandimos para 2D
if y_test_pred.ndim == 1:
    y_test_pred = y_test_pred[:, np.newaxis]

# Se for 2D (batch_size, timesteps), pegamos o último timestep
if y_test_pred.ndim == 2:
    y_test_pred_final = y_test_pred[:, -1]  #  Sem `:` no final, pois já é 1D
else:
    y_test_pred_final = y_test_pred[:, -1, :]  #  Apenas se for 3D

y_test_pred_labels = (y_test_pred_final > 0.5).astype(int)

y_test_true = y_test.flatten()
accuracy = np.mean(y_test_pred_labels == y_test_true)
print(f"\nAccuracy final no conjunto de teste: {accuracy:.4f}")

# Criar DataFrame com Expected vs Predicted
df_results = pd.DataFrame({
    "expected_value": y_test_true,
    "predicted_value_raw": y_test_pred_final.flatten(),  # Valor original antes do arredondamento
    "predicted_value": y_test_pred_labels.flatten()  # Valor final binário (0 ou 1)
})

# Mostrar as previsões para comparação
print("\nComparação entre valores esperados e previstos:")
print(df_results)

**Previsão do melhor modelo para o dataset disponibilizado pelo professor**

In [ ]:
# Testar Modelo Final
y_test_pred2 = final_rnn.predict(X_eval_final)

print(f"Formato de y_test_pred2: {y_test_pred2.shape}")  # Debug

# Se for 1D, expandimos para 2D
if y_test_pred2.ndim == 1:
    y_test_pred2 = y_test_pred2[:, np.newaxis]

# Se for 2D (batch_size, timesteps), pegamos o último timestep
if y_test_pred2.ndim == 2:
    y_test_pred_final2 = y_test_pred2[:, -1]  #  Sem `:` no final, pois já é 1D
else:
    y_test_pred_final2 = y_test_pred2[:, -1, :]  #  Apenas se for 3D

y_test_pred_labels2 = (y_test_pred_final2 > 0.5).astype(int)


**Criação do Ficheiro CSV com a previsão final para o dataset disponibilizado pelo professor**

In [ ]:
# Generate IDs for each prediction in the format D2-1, D2-2, ...
ids = [f"D2-{i+1}" for i in range(len(y_test_pred_labels2))]

# Map 0 to "Human" and 1 to "AI"
labels = ["Human" if pred == 0 else "AI" for pred in y_test_pred_labels2.flatten()]

# Create a DataFrame with ID and Label columns
df_predictions = pd.DataFrame({
    "ID": ids,
    "Label": labels
})

# Save the predictions to a CSV file using a tab separator to match the exact format
df_predictions.to_csv("classify_output_datasets/dataset2_outputs_rnn_model.csv", sep="\t", index=False)

print("\nPredictions saved to dataset2_outputs_rnn_model.csv successfully!")

### **Análise de resultados da melhor combinação encontrada**

**Melhor combinação encontrada: {'epochs': 5, 'batch_size': 8, 'learning_rate': 0.01, 'momentum': 0.8, 'bptt_trunc': 6} com accuracy 0.8929**

**Treino com dataset: gpt_vs_human**

- Durante o treino: 0.87 - 0.9

- Para dataset1: 0.60

- Para dataset2: 0.8 - 1